In [1]:
import boto3, os, logging, botocore
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

import sys
import os

import s3fs
import h5py

import time


import tensorflow as tf
#from tensorflow.keras import layers, models
print(tf.__version__)
print(tf.__file__)
print(tf.config.list_physical_devices('GPU'))
#print(tf.keras.__version__)
import keras
from keras import layers, models
print(keras.__version__)

boto3.set_stream_logger('boto3', logging.INFO)
s3 = boto3.resource('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED)) 
bucket = s3.Bucket('nrel-pds-porotomo')
cwd = os.getcwd()

2024-12-04 21:22:39.531993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-04 21:22:39.548724: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-04 21:22:39.553256: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 21:22:39.565487: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.17.0
/home/acrook/anaconda3/lib/python3.12/site-packages/tensorflow/__init__.py
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
3.4.1


I0000 00:00:1733372562.835411   14192 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733372562.907550   14192 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1733372562.907639   14192 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [5]:
def preprocess_hdf5_raw_chunks(
    filepaths,
    output_file,
    chunk_size_seconds,
    sampling_rate
):
    """
    Preprocess HDF5 data from S3 by downloading entire files, splitting into 1-second chunks,
    and partitioning into spatial chunks. Drops last 21 spatial points for divisibility by 100.

    Parameters:
        filepaths (list): List of S3 file paths for the input HDF5 files.
        output_file (str): Name of the output HDF5 file for raw data chunks.
        chunk_size_seconds (int): Length of each time chunk in seconds.
        sampling_rate (int): Sampling rate of the data in Hz.
    """
    s3 = s3fs.S3FileSystem(anon=True)
    chunk_size_samples = chunk_size_seconds * sampling_rate

    # Create the output HDF5 file
    with h5py.File(output_file, 'w') as h5_file:
        chunk_idx = 0

        for filepath in filepaths:
            print(f"Processing file: {filepath}")
            
            # Step 1: Download file locally
            start_download = time.time()
            local_file = os.path.basename(filepath)
            with s3.open(filepath, 'rb') as s3_file, open(local_file, 'wb') as local_fp:
                local_fp.write(s3_file.read())
            download_time = time.time() - start_download
            print(f"Downloaded file locally in {download_time:.2f} seconds.")
            
            # Step 2: Open the downloaded file and load data
            start_load = time.time()
            with h5py.File(local_file, 'r') as f:
                das_data = f['das']
                num_samples, num_spatial_points = das_data.shape
                truncated_data = das_data[:, :num_spatial_points - 21]  # Drop last 21 spatial points
            load_time = time.time() - start_load
            print(f"Loaded and truncated dataset in {load_time:.2f} seconds.")


            # Step 3: Process data into chunks
            for start in range(0, num_samples, chunk_size_samples):
                end = min(start + chunk_size_samples, num_samples)
                time_chunk = truncated_data[start:end, :]  # Extract 1-second chunk

                # Partition into spatial chunks of 100 points
                num_chunks = time_chunk.shape[1] // 100
                for i in range(num_chunks):
                    spatial_chunk = time_chunk[:, i * 100:(i + 1) * 100]

                    # Save the spatial chunk with the format chunk_timeindex_spatialindex
                    chunk_name = f'chunk_{chunk_idx}_{start // chunk_size_samples}_{i}'
                    print(chunk_name)
                    h5_file.create_dataset(chunk_name, data=spatial_chunk)
            chunk_idx += 1

            '''
            # Step 3: Process data into chunks
            for start in range(0, num_samples, chunk_size_samples):
                end = min(start + chunk_size_samples, num_samples)
                time_chunk = truncated_data[start:end, :]  # Extract 1-second chunk

                # Partition into spatial chunks of 100 points
                num_chunks = time_chunk.shape[1] // 100
                for i in range(num_chunks):
                    spatial_chunk = time_chunk[:, i * 100:(i + 1) * 100]

                    # Save the spatial chunk
                    h5_file.create_dataset(f'chunk_{chunk_idx}', data=spatial_chunk)
                    chunk_idx += 1'''

            # Step 4: Cleanup local file
            os.remove(local_file)
            print(f"Removed local copy of file {local_file}.")

            print(f"Finished processing file: {filepath}")

    print("All files processed and saved successfully.")


In [3]:
keys = ['PoroTomo_iDAS16043_160315083149.h5',
        'PoroTomo_iDAS16043_160315083219.h5',
        'PoroTomo_iDAS16043_160315000019.h5',
       'PoroTomo_iDAS16043_160315014019.h5',
       'PoroTomo_iDAS16043_160315034519.h5',
       'PoroTomo_iDAS16043_160315023049.h5',
       'PoroTomo_iDAS16043_160315023119.h5',
       'PoroTomo_iDAS16043_160315025349.h5',
       'PoroTomo_iDAS16043_160315031649.h5',
       'PoroTomo_iDAS16043_160315032449.h5',]

#keys = ['PoroTomo_iDAS16043_160315083149.h5']

filepaths = ['s3://nrel-pds-porotomo/DAS/H5/DASH/20160315/' + key for key in keys]
print(filepaths)

['s3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315083149.h5', 's3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315083219.h5', 's3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315000019.h5', 's3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315014019.h5', 's3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315034519.h5', 's3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315023049.h5', 's3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315023119.h5', 's3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315025349.h5', 's3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315031649.h5', 's3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315032449.h5']


In [6]:
preprocess_hdf5_raw_chunks(
    filepaths=filepaths,
    output_file="raw_data.h5",
    chunk_size_seconds=1,
    sampling_rate=1000
)


Processing file: s3://nrel-pds-porotomo/DAS/H5/DASH/20160315/PoroTomo_iDAS16043_160315083149.h5
Downloaded file locally in 92.44 seconds.
Loaded and truncated dataset in 0.21 seconds.
chunk_0_0_0
chunk_0_0_1
chunk_0_0_2
chunk_0_0_3
chunk_0_0_4
chunk_0_0_5
chunk_0_0_6
chunk_0_0_7
chunk_0_0_8
chunk_0_0_9
chunk_0_0_10
chunk_0_0_11
chunk_0_0_12
chunk_0_0_13
chunk_0_0_14
chunk_0_0_15
chunk_0_0_16
chunk_0_0_17
chunk_0_0_18
chunk_0_0_19
chunk_0_0_20
chunk_0_0_21
chunk_0_0_22
chunk_0_0_23
chunk_0_0_24
chunk_0_0_25
chunk_0_0_26
chunk_0_0_27
chunk_0_0_28
chunk_0_0_29
chunk_0_0_30
chunk_0_0_31
chunk_0_0_32
chunk_0_0_33
chunk_0_0_34
chunk_0_0_35
chunk_0_0_36
chunk_0_0_37
chunk_0_0_38
chunk_0_0_39
chunk_0_0_40
chunk_0_0_41
chunk_0_0_42
chunk_0_0_43
chunk_0_0_44
chunk_0_0_45
chunk_0_0_46
chunk_0_0_47
chunk_0_0_48
chunk_0_0_49
chunk_0_0_50
chunk_0_0_51
chunk_0_0_52
chunk_0_0_53
chunk_0_0_54
chunk_0_0_55
chunk_0_0_56
chunk_0_0_57
chunk_0_0_58
chunk_0_0_59
chunk_0_0_60
chunk_0_0_61
chunk_0_0_62
chunk_0

In [ ]:

import h5py

# File path for the normalized data
normalized_file_path = "normalized_raw_data.h5"

# Open the HDF5 file and inspect its structure
with h5py.File(normalized_file_path, 'r') as f:
    # Print all keys and subgroups
    def print_structure(name, obj):
        print(name)
    f.visititems(print_structure)

In [ ]:
'''import h5py
import numpy as np
import matplotlib.pyplot as plt

# File path to your normalized data
normalized_file_path = "normalized_raw_data.h5"

# Function to plot chunks of data
def plot_chunks(chunk_size=30, num_spatial_points=5, ylim=[-2, 2]):
    # Open the HDF5 file containing your chunks
    with h5py.File(normalized_file_path, 'r') as f:
        # List all datasets (chunks) in the file
        chunk_names = [name for name in f.keys() if name.startswith('chunk_')]
        
        # Ensure we plot in batches of 'chunk_size'
        for i in range(0, len(chunk_names), chunk_size):
            batch_chunks = chunk_names[i:i + chunk_size]
            
            # Create a new figure for each batch of chunks
            fig, axs = plt.subplots(len(batch_chunks) * num_spatial_points, 1, figsize=(12, len(batch_chunks) * num_spatial_points * 2), sharex=True, sharey=True)
            fig.subplots_adjust(hspace=0.5)
            
            for j, chunk_name in enumerate(batch_chunks):
                # Load the chunk data from the HDF5 file
                chunk_data = f[chunk_name][:]
                
                # Loop through spatial points and plot
                for spatial_index in range(num_spatial_points):
                    # Calculate the corresponding row index for the subplot
                    row = j * num_spatial_points + spatial_index
                    
                    # Create a plot for the spatial point
                    ax = axs[row]
                    ax.plot(np.array(chunk_data[:, spatial_index]))
                    ax.set_title(f'{chunk_name} - Spatial Point {spatial_index}')
                    ax.set_ylim(ylim)
                    ax.set_xlabel('Time (samples)')
                    ax.set_ylabel('Amplitude')
            
            plt.show()

# Call the plotting function
plot_chunks()'''
